In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)

In [ ]:
%%px
from mpi4py import MPI
comm=MPI.COMM_WORLD

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

from NavierStokesSIMPLE import NavierStokes
ngsglobals.msg_level = 0

In [ ]:
%%px

shape = Rectangle(2,0.41).Circle(0.2,0.2,0.05).Reverse().Face()
shape.edges.name="wall"
shape.edges.Min(X).name="inlet"
shape.edges.Max(X).name="outlet"
# Draw (shape);

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.07, comm=comm)).Curve(3)
if comm.rank==1:
    Draw (mesh);

In [ ]:
%%px
timestep = 0.001
navstokes = NavierStokes (mesh, nu=0.001, order=5, timestep = timestep,
                              inflow="inlet", outflow="outlet", wall="wall|cyl",
                              uin=CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) ))
                              

navstokes.SolveInitial()

In [ ]:
%%px
if comm.rank==1:
    scene = Draw (Norm(navstokes.velocity), mesh, "velocity", order=3)

tend = 5
t = 0
cnt = 0
with TaskManager():
    while t < tend:
        navstokes.DoTimeStep()
        t = t+timestep
        cnt = cnt+1
        if cnt % 100 == 0:
            printmaster ("t = ", t) # , "  ", end='\r')
            if comm.rank==1:
                scene.Redraw()